In [1]:
#Import dependencies
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import re

In [4]:
movies_df=pd.read_csv("data/updated_movies.csv")

In [12]:
samples_movie_df=movies_df.loc[0:20000,:]
samples_movie_df['feature']=samples_movie_df['genres']+' '+samples_movie_df['avg_user_rating'].round(4).map(str)#+' '+samples_movie_df['year'].map(str)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
samples_movie_df

,movieId,title,genres,avg_user_rating,year,comb,feature
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,3.893708,1995,Adventure Animation Children Comedy Fantasy 3....,Adventure Animation Children Comedy Fantasy 3....
1,2,Jumanji (1995),Adventure Children Fantasy,3.251527,1995,Adventure Children Fantasy 3.2515 1995,Adventure Children Fantasy 3.2515
2,3,Grumpier Old Men (1995),Comedy Romance,3.142028,1995,Comedy Romance 3.142 1995,Comedy Romance 3.142
3,4,Waiting to Exhale (1995),Comedy Drama Romance,2.853547,1995,Comedy Drama Romance 2.8535 1995,Comedy Drama Romance 2.8535
4,5,Father of the Bride Part II (1995),Comedy,3.058434,1995,Comedy 3.0584 1995,Comedy 3.0584
...,...,...,...,...,...,...,...
19996,103673,Gitmo (2005),Documentary,2.666667,2005,Documentary 2.6667 2005,Documentary 2.6667
19997,103676,My Avatar and Me (Min Avatar og mig) (2010),Documentary,3.250000,2010,Documentary 3.25 2010,Documentary 3.25
19998,103681,"Blood Beast Terror, The (1968)",Crime Horror Mystery,2.142857,1968,Crime Horror Mystery 2.1429 1968,Crime Horror Mystery 2.1429
19999,103683,Othello (1981),Drama,3.333333,1981,Drama 3.3333 1981,Drama 3.3333


In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

cv = CountVectorizer()
count_matrix = cv.fit_transform(samples_movie_df['feature'])
#print(count_matrix)
#creating a similarity score matrix
sim = cosine_similarity(count_matrix)
print(sim)
#print(movies_df['comb'])

[[1.         0.61237244 0.23570226 ... 0.         0.         0.18257419]
 [0.61237244 1.         0.         ... 0.         0.         0.2236068 ]
 [0.23570226 0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.31622777]
 [0.18257419 0.2236068  0.         ... 0.         0.31622777 1.        ]]


In [24]:
samples_movie_df.head()

,movieId,title,genres,avg_user_rating,year,comb,feature
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,3.893708,1995,Adventure Animation Children Comedy Fantasy 3....,Adventure Animation Children Comedy Fantasy 3....
1,2,Jumanji (1995),Adventure Children Fantasy,3.251527,1995,Adventure Children Fantasy 3.2515 1995,Adventure Children Fantasy 3.2515
2,3,Grumpier Old Men (1995),Comedy Romance,3.142028,1995,Comedy Romance 3.142 1995,Comedy Romance 3.142
3,4,Waiting to Exhale (1995),Comedy Drama Romance,2.853547,1995,Comedy Drama Romance 2.8535 1995,Comedy Drama Romance 2.8535
4,5,Father of the Bride Part II (1995),Comedy,3.058434,1995,Comedy 3.0584 1995,Comedy 3.0584


In [25]:
# Build a 1-dimensional array with movie titles
indices = pd.Series(samples_movie_df.index, index=samples_movie_df['title'])

# Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21] 
    movie_indices = [i[0] for i in sim_scores]
    return list(samples_movie_df.iloc[movie_indices[0:10]]["title"])

In [26]:
import pickle

In [27]:
#Exporting similariy scores into a .pickl file for use in front-end
pickle.dump(sim,open("genrepredictor-similaritymatrix.pckl","wb"))

In [28]:
#Exporting dataframe into a .pickl file for use in front-end
pickle.dump(samples_movie_df,open("movie_df.pckl","wb"))

In [29]:
#Loading similarity-scores.pickl into variable
loaded_sim=pickle.load(open("genrepredictor-similaritymatrix.pckl","rb"))

In [30]:
#Loading dataframe.pickl into variable
loaded_df=pickle.load(open("movie_df.pckl","rb"))

In [31]:
#Running model to predict score based on movie
str(genre_recommendations('Waiting to Exhale (1995)'))

'[\'Fallen Angels (Duo luo tian shi) (1995)\', \'Rat (2000)\', \'30 Years to Life (2001)\', "Love and Anarchy (Film d\'amore e d\'anarchia, ovvero \'stamattina alle 10 in via dei Fiori nella nota casa di tolleranza...\') (1973)", \'Wife vs. Secretary (1936)\', \'Surviving Desire (1991)\', \'Beautiful Joe (2000)\', \'Monday Morning (Lundi matin) (2002)\', \'My Bollywood Bride (2006)\', \'Only Game in Town, The (1970)\']'